<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install kaggle

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [6]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 35.3MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 60.3MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 67.7MB/s]
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 125MB/s]


In [7]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip



Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.feature_selection as fe
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.feature_selection import RFECV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
#import category_encoders as ce


In [8]:
# DO NOT USE
# date as index
'''df_train = pd.read_csv('train_features.csv',index_col='date_recorded', 
              parse_dates=True, infer_datetime_format=True)
df_test = pd.read_csv('test_features.csv', index_col='date_recorded',
                     parse_dates=True, infer_datetime_format=True)
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape'''


((59400, 39), (14358, 39), (59400, 2))

In [8]:
# id as index
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape


((59400, 39), (14358, 39), (59400, 2), (14358, 2))

In [9]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        df_train, df_label, test_size=0.25, random_state=42, shuffle=True)

X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape

((44550, 39), (14850, 39), (44550, 2), (14850, 2))

In [0]:
'''pd.set_option('display.max_columns', None)

#df_str=X_train_val.select_dtypes(exclude=['int','float']).copy()
df_str=X_train_val.select_dtypes(object).copy()
df_str.head(3)'''


In [0]:
# As observations increase, more features will fail to pass the threshold
# At 50,000+ observation, passing threshold drops to 0.1

#remove all features that are either one or zero (on or off) in more than 80% of the samples.
#threshold = 0.8 * (1- .8)

import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.01):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
def wrangle(X):
  
  X_num = pd.DataFrame()
  # get all continuous value column
  X_num = X._get_numeric_data().copy()
  #X_num['construction_year'] = X.construction_year.copy()
  X_str = X.select_dtypes(object).copy()
  
  for col_nan in X_str.columns[X_str.isnull().any()].tolist():
    X_str[col_nan].fillna('other', inplace=True)
    X_str[col_nan] = X_str[col_nan].replace('other', 'otr_'+col_nan)
  
  # funder -- comment out 0.717-0.715
  X_str.funder = X_str.funder.replace('0', 'missing')
  X_dummies = pd.get_dummies(X_str.funder)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  # basin threshold0.02(0.720 - 0.718) 
  X_dummies = pd.get_dummies(X_str.basin)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # region threshold0.02(0.719 - 0.708), 0.01(0.716-0.714) 
  X_dummies = pd.get_dummies(X_str.region)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # lga -- 0.717 - 0.710
  X_dummies = pd.get_dummies(X_str.lga)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # scheme_management 0.714 - 0.711
  '''X.scheme_management=X.scheme_management.replace('Other','other_scheme')
  X.scheme_management=X.scheme_management.replace('None','other_scheme')
  X_dummies = pd.get_dummies(X.scheme_management)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')'''
  
  
  # extraction_type_class 0.714 - 0.713
  X_str.extraction_type_class.replace('other','otr_extr',inplace=True)
  X_dummies = pd.get_dummies(X_str.extraction_type_class)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  # management 0.713 - 0.708
  X_str.management=X_str.management.replace('unknown','unkwn_mgmt')
  X_str.management=X_str.management.replace('other','otr_mgmt')
  X_dummies = pd.get_dummies(X_str.management)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  # payment
  X_str.payment=X_str.payment.replace('unknown','unkwn_pmt')
  X_dummies = pd.get_dummies(X_str.payment)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # quality_group
  X_str.quality_group=X_str.quality_group.replace('unknown','unkwn_qual_grp')
  X_dummies = pd.get_dummies(X_str.quality_group)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # quantity
  X_str.quantity=X_str.quantity.replace('unknown','unknown_quantity')
  X_dummies = pd.get_dummies(X_str.quantity)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # source
  X_dummies = pd.get_dummies(X_str.source_type)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  # waterpoint_type
  X_str.waterpoint_type=X_str.waterpoint_type.replace('other','otr_wtr')
  X_dummies = pd.get_dummies(X_str.waterpoint_type)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  #y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
  #y = y.values
  
  
  return X_num
  

In [17]:
%%time

# Baseline metric - numeric only - test_score=0.55
X = wrangle(X_train_val)
y = y_train_val.status_group.values


scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None, max_iter=1500), 
                        X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",59.587724,0.041078,0.728907,0.727600
1,"LogisticRegression(C=1.0, class_weight=None, d...",60.835439,0.036753,0.723838,0.726397
2,"LogisticRegression(C=1.0, class_weight=None, d...",60.597925,0.036413,0.727322,0.730346


In [0]:
# NOT USE - for understanding variance_threshold only
# same as variance threshold function
def noise_filter(df, col, threshold=0.01):
  X_dummies=pd.DataFrame()
  X_dummies[col]=df[col].value_counts(normalize=True)

  for noise_cat in X_dummies.loc[(X_dummies[col] < threshold)].index.values:
    df[col].replace(noise_cat, 'noise', inplace=True)

  #df_str['funder'].replace('0', 'noise', inplace=True)
  #df_str['funder'].replace(np.nan,'noise' ,inplace=True)

  return pd.get_dummies(df[col])

dummy_filter = noise_filter(df_str, 'funder' ,0.01)
dummy_filter=dummy_filter.drop(['0','noise'], axis=1)
dummy_filter.shape

In [21]:
%%time
# Define an estimator and param_grid
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif,mutual_info_classif

#X = wrangle(X_train_val)
#y = y_train_val.status_group

X = wrangle(X_test)
y = y_test.status_group

pipe = make_pipeline(
    RobustScaler(), 
    #fe.SelectKBest(mutual_info_classif), #slow 0.58
    fe.SelectKBest(f_classif),
    LogisticRegression(max_iter=1000))

param_grid = {
    'selectkbest__k': [10,30,50, 'all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [.01, .1, 1.0, 10.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)

gs.fit(X, y)
validation_score = gs.best_score_
print('X shape=', X.shape)
print('Cross-Validation Accuracy Score:', validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed:  3.5min finished


('X shape=', (14850, 132))
('Cross-Validation Accuracy Score:', 0.7358922558922559)
()
('Best estimator:', Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k='all', score_func=<function f_classif at 0x7f963bed07d0>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]))
()
Best result from grid search of 64 parameter combinations
CPU times: user 3min 24s, sys: 33.8 s, total: 3min 58s
Wall time: 3min 30s


In [0]:
#Which features were selected?


selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X.columns.values
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


In [0]:
from sklearn.ensemble import RandomForestRegressor as RFR

X=wrangle(X_train_val)
names = X.columns

y = y_train_val.status_group.values

y = pd.DataFrame(y).replace(['non functional','functional','functional needs repair'],[1,2,3])
y = y.values

# Fitting a random forest regression
rfr = RFR().fit(X, y)

# Creating scores from feature_importances_ ranking (some randomness here)
rfr_scores = pd.Series(data=rfr.feature_importances_, name='RFR', index=names)

rfr_scores

In [0]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

X= wrangle(X_train_val)

y= y_train_val.status_group

#y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
#y = y.values

X_scaled = RobustScaler().fit_transform(X)

# Lasso can't handle multi-class label.
#rfe = RFECV(Lasso(), scoring='accuracy', cv=3)
rfe = RFECV(LogisticRegression(), scoring='accuracy', cv=3)
X_subset = rfe.fit_transform(X_scaled, y)

all_names = X.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)


In [22]:
# Predict with X_test features
#X=wrangle(X_test)
y_pred = gs.predict(X)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test.status_group, y_pred)
print('Test Score:', test_score)

('Test Score:', 0.7404713804713805)


In [23]:
df_testX=wrangle(df_test)
X.shape, df_testX.shape

((14850, 132), (14358, 134))

In [0]:
t = pd.DataFrame()
for name in selected_names:
  t[name]=df_testX[name]

In [0]:
y_pred = gs.predict(t)

In [0]:
# Predict with X_test features
X=wrangle(df_test)
y_pred = gs.predict(X)


In [38]:
#submission as format as df_sample
y_submission = pd.DataFrame()

y_submission['id'] = df_test.index
y_submission['status_group']=y_pred
y_submission.shape

(14358, 2)

In [0]:
from google.colab import files

y_submission.to_csv('data_all.csv', index=False)
files.download("data_all.csv")